# On Demand water maps via HyP3-watermap

This notebook will leverage either [ASF Search -- Vertex](https://search.asf.alaska.edu/#/) or the
[asf_search](https://github.com/asfadmin/Discovery-asf_search) Python package, and the
[HyP3 SDK](https://hyp3-docs.asf.alaska.edu/using/sdk/), to request On Demand surface water extent maps
from the custom [hyp3-watermap](https://hyp3-watermap.asf.alaska.edu) HyP3 deployment.

Water maps are generated from Sentinel-1 SLCs or GRDs by:
1. Applying Radiometric Terrain Correction (RTC)
2. Creating initial VV- and VH-based water maps using a thresholding approach
3. Refining the initial VV- and VH-based water maps using fuzzy logic
4. Combining the refined VV- and VH-based water maps into a final water map

For more information on the methods, or to modify the water map methods and process them locally, see the
[water-extent-map.ipynb](water-extent-map.ipynb) notebook.

## 0. Initial setup

Import and setup some helper functions for this notebook.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def wkt_input():
    wkt = widgets.Textarea(
        placeholder='WKT of search area',
        value='POLYGON((-91.185 36.6763,-86.825 36.6763,-86.825 38.9176,-91.185 38.9176,-91.185 36.6763))',
        layout=widgets.Layout(width='100%'),
    )
    display(wkt)
    return wkt

def file_ids_input():
    file_ids = widgets.Textarea(
        placeholder='copy-paste Sentinel-1 granule names or file ids here (One granule or id per line)',
        layout=widgets.Layout(width='100%', height='12em'),
    )
    display(file_ids)
    return file_ids

## 1. Search for Sentinel-1 scenes to process

You can search for Sentinel-1 scenes with either [ASF Search -- Vertex](https://search.asf.alaska.edu/#/) or the
[asf_search](https://github.com/asfadmin/Discovery-asf_search) Python package. Vertex provides an interactive,
feature rich experience, while `asf_search` allows searching programmatically and mirrors the vertex interface
as best it can. Section 1.1 describes using Vertex and Section 1.2 describes using `asf_search`.

*Note: only 1.1 or 1.2 needs to be executed to run this notebook.*

### 1.1 Search for  Sentinel-1 scenes in Vertex

Requesting water map products from the custom HyP3-watermap deployment looks very similar to
[requesting On Demand RTC products](https://storymaps.arcgis.com/stories/2ead3222d2294d1fae1d11d3f98d7c35),
**except** instead of adding scenes to your On Demand queue, you'll:
1. add the scenes to your Downloads cart
   ![add to cart](https://user-images.githubusercontent.com/7882693/122344682-85acc800-cef3-11eb-8337-5a356b722c12.png)


2. open the Downloads Cart and select "Copy File Ids", and
   ![image](https://user-images.githubusercontent.com/7882693/122345160-04096a00-cef4-11eb-8c27-892329293e4d.png)


3. paste the file ids into the text area that will appear below the next cell.

**Note:** Water maps currently require the Sentinel-1 source granules to be SLCs (preferred) or High-Res GRDs,
acquired using the IW beam mode, with both VV and VH polarizations. You can use the
[example search](https://search.asf.alaska.edu/#/?beamModes=IW&polarizations=VV%2BVH&productTypes=SLC&zoom=6.190&center=-91.993,33.963&polygon=POLYGON((-91.185%2036.6763,-86.825%2036.6763,-86.825%2038.9176,-91.185%2038.9176,-91.185%2036.6763))&start=2021-05-30T00:00:00Z&resultsLoaded=true&granule=S1A_IW_GRDH_1SDV_20210607T234810_20210607T234835_038241_04834F_4BB6-GRD_HD&end=2021-06-07T23:59:59Z)
or jump-start your search in Vertex (with the required parameters already set) by following [this link](https://search.asf.alaska.edu/#/?dataset=Sentinel-1&productTypes=SLC&beamModes=IW&polarizations=VV%2BVH).

In [ ]:
file_ids = file_ids_input()

In [ ]:
all_granules = [f.strip().split('-')[0] for f in file_ids.value.splitlines()]
display(sorted(all_granules))

### 1.2 Search for Sentinel-1 scenes with `asf_search`

We'll use the geographic search functionality of `asf_search` to perform a search over an Area of
Interest (AOI) represented as [Well-Known Text (WKT)](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry).

You can use the example WKT, or copy and paste your AOI's WKT, in the text area that will appear below the next cell.

In [ ]:
wkt = wkt_input()

Water maps currently require the Sentinel-1 source granules to be SLCs (preferred) or High-Res GRDs,
acquired using the IW beam mode, with both VV and VH polarizations. The next cell performs a search over your AOI,
with these parameters set.

*Note: You will likely want to edit the `start` and `end` parameters.*

In [ ]:
import asf_search
from asf_search.constants import SENTINEL1, SLC, IW, VV_VH

search_results = asf_search.geo_search(
    platform=[SENTINEL1],
    processingLevel=[SLC],
    beamMode=[IW],
    polarization=[VV_VH],
    intersectsWith=wkt.value,
    start='2021-05-30',
    end='2021-06-08',
)
all_granules = {result.properties['sceneName'] for result in search_results}
display(sorted(all_granules))

## 2. Request water maps from HyP3-watermap

### 2.1 Connect to the HyP3-watermap deployment

Use the HyP3 SDK to connect to the custom deployment with your [NASA Earthdata login](https://urs.earthdata.nasa.gov/).

In [ ]:
import hyp3_sdk

hyp3_watermap = hyp3_sdk.HyP3('https://hyp3-watermap.asf.alaska.edu', prompt=True)

### 2.2 Specify the custom water map parameters

Below is a dictionary representation of the possible customization options for a water-map job.
Importantly, this definition will be applied to each granule in our search results, so these
options will be used with each job we submit.

You may change any or all of them, and in particular, you will likely want to use the
`name` parameter to group each "batch" of jobs together and easily find them later.

In [ ]:
job_definition = {
    'name': 'water-map-example',
    'job_type': 'WATER_MAP',
    'job_parameters': {
        'resolution': 30,
        'speckle_filter': True,
        'max_vv_threshold': -15.5,
        'max_vh_threshold': -23.0,
        'hand_threshold': 15.0,
        'hand_fraction': 0.8,
        'membership_threshold': 0.45,
    }
}

### 2.3 Submit the jobs to the custom HyP3-watermap deployment

Using the job definition as defined above (make sure you run the cell!), this will submit a job for
each granule in the search results.

In [ ]:
import copy

prepared_jobs = []
for granule in all_granules:
    job = copy.deepcopy(job_definition)
    job['job_parameters']['granules'] = [granule]
    prepared_jobs.append(job)

jobs = hyp3_watermap.submit_prepared_jobs(prepared_jobs)

Once the jobs are submitted, you can watch for them to complete (it will take ~30 min for all jobs to finish).

In [ ]:
jobs = hyp3_watermap.watch(jobs)

Or, you can come back later and find your jobs by name, and make sure they're finished

In [ ]:
jobs = hyp3_watermap.find_jobs(name='water-map-example')
jobs = hyp3_watermap.watch(jobs)

Once all jobs are complete, you can download the products for each successful job

In [ ]:
jobs.download_files('data/')

## Notes on viewing/evaluating the water map products

* All GeoTIFFs in the RTC products are Cloud-Optimized, including the water map files `*_WM.tif`, and will have overviews/pyramids.

  **This means the `*_WM.tif`'s appear to have a significantly higher water extent than they do in reality until you zoom in.**